# Generate JSON

Generate JSON with metadata for the collections to be published on `data.lsdb.io`.

In [ ]:
import os
import json
import lsdb
import subprocess

from pathlib import Path

In [ ]:
INSTRUMENT = os.environ["INSTRUMENT"]
RUN = os.environ["RUN"]
VERSION = os.environ["VERSION"]
COLLECTION = os.environ["COLLECTION"]
OUTPUT_DIR = Path(os.environ["OUTPUT_DIR"])

print(f"INSTRUMENT: {INSTRUMENT}")
print(f"RUN: {RUN}")
print(f"VERSION: {VERSION}")
print(f"COLLECTION: {COLLECTION}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

hats_dir = OUTPUT_DIR / "hats" / VERSION

In [ ]:
def get_directory_size(path):
    SIZE_UNITS = {"G": "GiB", "M": "MiB", "K": "KiB", "T": "TiB"}
    result = subprocess.run(
        ["du", "-sh", path], capture_output=True, text=True, check=True
    )
    size_str = result.stdout.split("\t")[0]
    size_unit = size_str[-1]
    return f"{size_str[:-1]} {SIZE_UNITS[size_unit]}"


def generate_json(catalog_name):
    catalog_path = hats_dir / catalog_name
    catalog = lsdb.read_hats(catalog_path)
    return {
        "label": f"{VERSION}/{catalog_name}",
        "name": f"{RUN} {VERSION} {catalog_name}",
        "description": f"DRP/{RUN}/{VERSION}/{COLLECTION} {catalog_name}",
        "urls": {"catalog": f"{hats_dir / catalog_name}"},
        "other_urls": [
            {
                "label": "Column descriptions",
                "url": "https://sdm-schemas.lsst.io/imsim.html",
            },
            {
                "label": "Jira Ticket",
                "url": f"https://rubinobs.atlassian.net/browse/{COLLECTION}",
            },
        ],
        "metadata": {
            "numRows": len(catalog),
            "numColumns": len(catalog.columns),
            "numPartitions": len(catalog.get_healpix_pixels()),
            "sizeOnDisk": get_directory_size(catalog_path),
        },
        "badges": ["Available only on USDF"],
    }

In [ ]:
nested_catalogs_json = [
    generate_json("dia_object_collection"),
    generate_json("object_collection"),
]
other_catalogs_json = [
    generate_json(catalog)
    for catalog in [
        #"dia_object_lc_x_ztf_dr22",
        #"dia_object_lc_x_ps1",
        #"object_lc_x_ztf_dr22",
        #"object_lc_x_ps1",
    ]
]

weekly_json = nested_catalogs_json + other_catalogs_json
with open(f"{VERSION}.json", "w") as file:
    json.dump(weekly_json, file)